In [1]:
class Passenger:
    def __init__(self):
        self.personID = None
        self.fromAirport = None
        self.toAirport = None
        self.preferredDepartureTime = None
        self.preferredDay = None
        self.preferredDepartureTimeRange = None  # Converted time to the range of 0 to 1008
        self.maxWaitTime = None  # Maximum time the passenger can wait


class Flight:
    def __init__(self):
        self.departureTime = None
        self.arrivalTime = None
        self.originCity = None
        self.destinationCity = None
        self.filledSeats = None
        self.avgWaitingTime = None
        self.profit = None


class Airport:
    def __init__(self):
        self.flights = []
        self.maxFlightsPerHour = None
        self.flightThresholds = []

In [110]:
# variables
totalFlights = 5
numberOfAirports = 3
totalTimeIntervals = 10


fixedCostPerHourOfFlight = 100
costPerSeatPerHourOfFlight = 50
numberOfSeatsPerFlight = 5

minimumFractionAirplaneFillesd = 0.8

timeFromOneAirportToAnother = 3 # 3 intervals


## while alltoing random values to number of people travelling per timeintervale, what is the max valu;e
maxPeoplePerTimeIntervale = 4

In [111]:
import numpy as np

# Create a synthetic dataset
airport_traffic = np.random.randint(0, high=maxPeoplePerTimeIntervale, size=(numberOfAirports, numberOfAirports, totalTimeIntervals))

# Set the diagonal values to 0 since flights from an airport to itself are not considered
# np.fill_diagonal(airport_traffic, 0)

print("Airport Traffic Dataset:")

for i in range(0, len(airport_traffic)):
    for j in range(0, len(airport_traffic)):
        for k in range (0 ,totalTimeIntervals):
            if (i==j):
                airport_traffic[i][j][k] = 0
print(airport_traffic)


Airport Traffic Dataset:
[[[0 0 0 0 0 0 0 0 0 0]
  [0 0 3 3 2 3 0 1 0 3]
  [1 3 3 1 0 0 0 0 1 1]]

 [[1 2 0 3 2 1 1 1 0 1]
  [0 0 0 0 0 0 0 0 0 0]
  [3 1 0 2 1 0 2 0 0 1]]

 [[0 3 2 1 2 3 2 0 1 3]
  [2 1 1 0 2 1 1 0 2 0]
  [0 0 0 0 0 0 0 0 0 0]]]


In [112]:
# ensuring for every airport there is atleast one flight in the stating
while True:
    iniitial_flights_positions = np.random.randint(0, numberOfAirports, size=totalFlights)
    
    # Check if all numbers between 0 and numberOfAirports - 1 are present
    if set(range(numberOfAirports)) <= set(iniitial_flights_positions):
        break

print("Initial flights positions:", iniitial_flights_positions)


Initial flights positions: [1 2 0 1 0]


In [113]:
airport_traffic_notChanged = airport_traffic.copy()
airport_traffic_notChanged

array([[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
        [0, 0, 3, 3, 2, 3, 0, 1, 0, 3],
        [1, 3, 3, 1, 0, 0, 0, 0, 1, 1]],

       [[1, 2, 0, 3, 2, 1, 1, 1, 0, 1],
        [0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
        [3, 1, 0, 2, 1, 0, 2, 0, 0, 1]],

       [[0, 3, 2, 1, 2, 3, 2, 0, 1, 3],
        [2, 1, 1, 0, 2, 1, 1, 0, 2, 0],
        [0, 0, 0, 0, 0, 0, 0, 0, 0, 0]]])

In [141]:
import numpy as np

airport_traffic = airport_traffic_notChanged.copy()
# print(airport_traffic)

flightLocations = iniitial_flights_positions 
flights_status = [{'currently': city,
                   'take_off_time': 0,
                   'landing_time': 0,
                   'from_city': None,
                   'to_city': None,
                   'seats_full': None,
                   'avg_wait_time': None,
                   'profit': None}
                  for city in iniitial_flights_positions]

schedule_array = [[] for _ in range(totalFlights)]
everyPossibleSchedule = []

def getSeatsLeft(airport_traffic, current_airport, dest_airport, time_instant): 
    seats_left = numberOfSeatsPerFlight
    for time_instant in range(0, time_instant + 1):
        if airport_traffic[current_airport][dest_airport][time_instant] > 0:
            demand = airport_traffic[current_airport][dest_airport][time_instant]
            if seats_left > demand: 
                airport_traffic[current_airport][dest_airport][time_instant] = 0
                seats_left -= demand
            else:
                airport_traffic[current_airport][dest_airport][time_instant] = demand - seats_left
                seats_left = 0
    return seats_left

def schedule_flights(time_instant, flights_status, schedule_array, airport_traffic):
    if time_instant == totalTimeIntervals:
        everyPossibleSchedule.append([flight_schedule[:] for flight_schedule in schedule_array])  # Append a copy
        return
    
    for flight_idx in range(len(flights_status)):
        if flights_status[flight_idx]['currently'] == -1:
            if flights_status[flight_idx]['landing_time'] == time_instant:
                flights_status[flight_idx]['currently'] = flights_status[flight_idx]['to_city']
        if flights_status[flight_idx]['currently'] == -1:
            continue
        
        flight_old = flights_status[flight_idx].copy()
            
        current_airport = flights_status[flight_idx]['currently']
        for dest_airport in range(numberOfAirports):
            if dest_airport == current_airport:
                continue
            
            seatsLeft = getSeatsLeft(airport_traffic, current_airport, dest_airport, time_instant)
            print(airport_traffic)
            print("Seats filled:", seatsLeft)
            if seatsLeft > minimumFractionAirplaneFillesd * numberOfSeatsPerFlight:
                continue
            
            flights_status[flight_idx]['currently'] = -1
            flights_status[flight_idx]['take_off_time'] = time_instant
            flights_status[flight_idx]['landing_time'] = time_instant + timeFromOneAirportToAnother
            flights_status[flight_idx]['from_city'] = current_airport
            flights_status[flight_idx]['to_city'] = dest_airport
            flights_status[flight_idx]['seats_full'] = numberOfSeatsPerFlight - seatsLeft
            flights_status[flight_idx]['profit'] =   costPerSeatPerHourOfFlight * (numberOfSeatsPerFlight - seatsLeft) - fixedCostPerHourOfFlight
            
            schedule_array[flight_idx].append(flights_status[flight_idx].copy())
            
            schedule_flights(time_instant + 1, flights_status, schedule_array, airport_traffic)
            
            flights_status[flight_idx] = flight_old
            schedule_array[flight_idx].pop()
            
            schedule_flights(time_instant + 1, flights_status, schedule_array, airport_traffic)

schedule_flights(0, flights_status, schedule_array, airport_traffic)

# Print everyPossibleSchedule to check the output
print(everyPossibleSchedule)


[[[0 0 0 0 0 0 0 0 0 0]
  [0 0 3 3 2 3 0 1 0 3]
  [1 3 3 1 0 0 0 0 1 1]]

 [[0 2 0 3 2 1 1 1 0 1]
  [0 0 0 0 0 0 0 0 0 0]
  [3 1 0 2 1 0 2 0 0 1]]

 [[0 3 2 1 2 3 2 0 1 3]
  [2 1 1 0 2 1 1 0 2 0]
  [0 0 0 0 0 0 0 0 0 0]]]
Seats filled: 4
[[[0 0 0 0 0 0 0 0 0 0]
  [0 0 3 3 2 3 0 1 0 3]
  [1 3 3 1 0 0 0 0 1 1]]

 [[0 2 0 3 2 1 1 1 0 1]
  [0 0 0 0 0 0 0 0 0 0]
  [3 1 0 2 1 0 2 0 0 1]]

 [[0 0 2 1 2 3 2 0 1 3]
  [2 1 1 0 2 1 1 0 2 0]
  [0 0 0 0 0 0 0 0 0 0]]]
Seats filled: 2
[[[0 0 0 0 0 0 0 0 0 0]
  [0 0 0 3 2 3 0 1 0 3]
  [1 3 3 1 0 0 0 0 1 1]]

 [[0 2 0 3 2 1 1 1 0 1]
  [0 0 0 0 0 0 0 0 0 0]
  [3 1 0 2 1 0 2 0 0 1]]

 [[0 0 2 1 2 3 2 0 1 3]
  [2 1 1 0 2 1 1 0 2 0]
  [0 0 0 0 0 0 0 0 0 0]]]
Seats filled: 2
[[[0 0 0 0 0 0 0 0 0 0]
  [0 0 0 0 2 3 0 1 0 3]
  [1 3 3 1 0 0 0 0 1 1]]

 [[0 2 0 3 2 1 1 1 0 1]
  [0 0 0 0 0 0 0 0 0 0]
  [3 1 0 2 1 0 2 0 0 1]]

 [[0 0 2 1 2 3 2 0 1 3]
  [2 1 1 0 2 1 1 0 2 0]
  [0 0 0 0 0 0 0 0 0 0]]]
Seats filled: 2
[[[0 0 0 0 0 0 0 0 0 0]
  [0 0 0 0 0 3 0 1 0 3]


In [142]:
airport_traffic

array([[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0, 0, 0, 0, 0]],

       [[0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0, 0, 0, 0, 0]],

       [[0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0, 0, 0, 0, 0]]])

In [143]:
iniitial_flights_positions

array([1, 2, 0, 1, 0])

In [146]:
schedule_flights(0,flights_status, schedule_array, airport_traffic)    

[[[0 0 0 0 0 0 0 0 0 0]
  [0 0 0 0 0 0 0 0 0 0]
  [0 0 0 0 0 0 0 0 0 0]]

 [[0 0 0 0 0 0 0 0 0 0]
  [0 0 0 0 0 0 0 0 0 0]
  [0 0 0 0 0 0 0 0 0 0]]

 [[0 0 0 0 0 0 0 0 0 0]
  [0 0 0 0 0 0 0 0 0 0]
  [0 0 0 0 0 0 0 0 0 0]]]
Seats filled: 5
[[[0 0 0 0 0 0 0 0 0 0]
  [0 0 0 0 0 0 0 0 0 0]
  [0 0 0 0 0 0 0 0 0 0]]

 [[0 0 0 0 0 0 0 0 0 0]
  [0 0 0 0 0 0 0 0 0 0]
  [0 0 0 0 0 0 0 0 0 0]]

 [[0 0 0 0 0 0 0 0 0 0]
  [0 0 0 0 0 0 0 0 0 0]
  [0 0 0 0 0 0 0 0 0 0]]]
Seats filled: 5
[[[0 0 0 0 0 0 0 0 0 0]
  [0 0 0 0 0 0 0 0 0 0]
  [0 0 0 0 0 0 0 0 0 0]]

 [[0 0 0 0 0 0 0 0 0 0]
  [0 0 0 0 0 0 0 0 0 0]
  [0 0 0 0 0 0 0 0 0 0]]

 [[0 0 0 0 0 0 0 0 0 0]
  [0 0 0 0 0 0 0 0 0 0]
  [0 0 0 0 0 0 0 0 0 0]]]
Seats filled: 5
[[[0 0 0 0 0 0 0 0 0 0]
  [0 0 0 0 0 0 0 0 0 0]
  [0 0 0 0 0 0 0 0 0 0]]

 [[0 0 0 0 0 0 0 0 0 0]
  [0 0 0 0 0 0 0 0 0 0]
  [0 0 0 0 0 0 0 0 0 0]]

 [[0 0 0 0 0 0 0 0 0 0]
  [0 0 0 0 0 0 0 0 0 0]
  [0 0 0 0 0 0 0 0 0 0]]]
Seats filled: 5
[[[0 0 0 0 0 0 0 0 0 0]
  [0 0 0 0 0 0 0 0 0 0]


In [147]:
everyPossibleSchedule

[[[{'currently': -1,
    'take_off_time': 0,
    'landing_time': 3,
    'from_city': 1,
    'to_city': 0,
    'seats_full': 1,
    'avg_wait_time': None,
    'profit': -50},
   {'currently': -1,
    'take_off_time': 3,
    'landing_time': 6,
    'from_city': 0,
    'to_city': 1,
    'seats_full': 3,
    'avg_wait_time': None,
    'profit': 50},
   {'currently': -1,
    'take_off_time': 6,
    'landing_time': 9,
    'from_city': 1,
    'to_city': 0,
    'seats_full': 4,
    'avg_wait_time': None,
    'profit': 100},
   {'currently': -1,
    'take_off_time': 9,
    'landing_time': 12,
    'from_city': 0,
    'to_city': 1,
    'seats_full': 3,
    'avg_wait_time': None,
    'profit': 50}],
  [{'currently': -1,
    'take_off_time': 1,
    'landing_time': 4,
    'from_city': 2,
    'to_city': 0,
    'seats_full': 3,
    'avg_wait_time': None,
    'profit': 50},
   {'currently': -1,
    'take_off_time': 4,
    'landing_time': 7,
    'from_city': 0,
    'to_city': 1,
    'seats_full': 2,
    

In [148]:
len(everyPossibleSchedule)

16

In [125]:
everyPossibleSchedule[0]

IndexError: list index out of range

In [107]:
a  = [1,2,3,4,5,6,7,8,9,10]

for i in a :
    i = i + 1
    
a

[1, 2, 3, 4, 5, 6, 7, 8, 9, 10]